<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#HMM-Analysis-for-Power-Sweep-Data" data-toc-modified-id="HMM-Analysis-for-Power-Sweep-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>HMM Analysis for Power Sweep Data</a></span></li></ul></div>

# HMM Analysis for Power Sweep Data

`project_path` should be the root folder of where all the Alazar data is stored 

In [ ]:
from AlazarPowerSweepData import *

%matplotlib qt

#project_path = r"G:\Shared drives\Quasiparticles_2\NBR07_Oct2022\test_power_sweep" 
project_path = str(input(r"Path to Measurement Run Root: "))

power_sweep_obj = AlazarPowerSweepData(project_path)

`createPdf` takes value `True` or `False`. Set to `True` if you want to visually see the downsampled IQ plots (determined by `avgTime`) to determine the **attenuation** below which the system goes non-linear. Set to `False` if the attenuation is known.

In [ ]:
createPdf = False

power_sweep_obj.process_Alazar_Data(avgTime=2,plots=createPdf)

`intTime` is the starting integration time provied to the HMM fits.
`SNRmin` is the minimum SNR cut off constraint to the HMM program.

In [ ]:
intTime=1
SNRmin=3

power_sweep_obj.start_HMM_fit(intTime=intTime, SNRmin= SNRmin)